# Carla Intro

Carla Python API is a Python library that allows you to control a Carla simulation environment.

You can find the official documentation [here](https://carla.readthedocs.io/en/latest/python_api/).

In [45]:
import carla
import time

## Carla client connetion

A connetion to the server will be created. From now, we can use the client to interact with the server.

> **Remember**: *Client* object is a proxy to the server. It is used to send commands to the server and receive responses.

In [46]:
client = carla.Client("localhost", 2000)

As a remote proxy, connection to the server is not immediate. It may take a few seconds to establish the connection.

In [47]:
client.set_timeout(15)
client.load_world("Town01") # a time consuming operation

In [48]:
client.set_timeout(15)
client.load_world("Town02")

In [49]:
print(f"Client version: {client.get_client_version()}")
print(f"Server version: {client.get_server_version()}")

Client version: 0.9.14
Server version: 0.9.14


## Configure the simulation

Client is also used to configure the simulation. We can set the weather, the time of the day, the map, etc.

### Change maps

More maps can be downloaded from [GitHub Releases](https://github.com/carla-simulator/carla/releases).

In [50]:
client.get_available_maps()

['/Game/Carla/Maps/Town01',
 '/Game/Carla/Maps/Town01_Opt',
 '/Game/Carla/Maps/Town02',
 '/Game/Carla/Maps/Town02_Opt',
 '/Game/Carla/Maps/Town03',
 '/Game/Carla/Maps/Town03_Opt',
 '/Game/Carla/Maps/Town04',
 '/Game/Carla/Maps/Town04_Opt',
 '/Game/Carla/Maps/Town05',
 '/Game/Carla/Maps/Town05_Opt',
 '/Game/Carla/Maps/Town10HD',
 '/Game/Carla/Maps/Town10HD_Opt']

`Town10HD_Opt` is a streamlined version of `Town10HD` designed to balance performance with environmental detail.

In [51]:
client.load_world("Town10HD_Opt")

### Set the weather

In [52]:
world = client.get_world()
world.set_weather(carla.WeatherParameters.WetNight)

In [53]:
weathers = [
    carla.WeatherParameters.CloudyNoon,
    carla.WeatherParameters.MidRainSunset,
    carla.WeatherParameters.HardRainNoon,
    carla.WeatherParameters.ClearSunset,
    carla.WeatherParameters.SoftRainNight,
    carla.WeatherParameters.Default,
]

for weather in weathers:
    world.set_weather(weather)
    print(f"Weather set to {weather}")
    time.sleep(5)

Weather set to WeatherParameters(cloudiness=60.000000, cloudiness=60.000000, precipitation=0.000000, precipitation_deposits=0.000000, wind_intensity=10.000000, sun_azimuth_angle=-1.000000, sun_altitude_angle=45.000000, fog_density=3.000000, fog_distance=0.750000, fog_falloff=0.100000, wetness=0.000000, scattering_intensity=1.000000, mie_scattering_scale=0.030000, rayleigh_scattering_scale=0.033100, dust_storm=0.000000)
Weather set to WeatherParameters(cloudiness=60.000000, cloudiness=60.000000, precipitation=60.000000, precipitation_deposits=60.000000, wind_intensity=60.000000, sun_azimuth_angle=-1.000000, sun_altitude_angle=15.000000, fog_density=3.000000, fog_distance=0.750000, fog_falloff=0.100000, wetness=0.000000, scattering_intensity=1.000000, mie_scattering_scale=0.030000, rayleigh_scattering_scale=0.033100, dust_storm=0.000000)
Weather set to WeatherParameters(cloudiness=100.000000, cloudiness=100.000000, precipitation=100.000000, precipitation_deposits=90.000000, wind_intensit

### Sync vs Async mode

In **synchronous** mode, the server waits for the client to send a tick command before moving to the next frame. This is useful when you want to control the simulation frame by frame.

In **asynchronous** mode, the server runs at its own pace. This is useful when you want to run the simulation as fast as possible.

![Sync vs Async](./img/sync-async.png)

In [54]:
world = client.get_world()
settings = world.get_settings()
settings.synchronous_mode = True
world.apply_settings(settings)

27519

In [55]:
count = 0
while True:
    world.tick()
    time.sleep(0.1)
    count += 1
    print(f"Ticked the server {count}", end="\r")

KeyboardInterrupt: 

In [56]:
settings.synchronous_mode = False
world.apply_settings(settings)
client.reload_world()

## Exercise: some config

In this exercise, we will connect to the server, configure the simulation, and run it for a few seconds.

The main goal is to get familiar with the Carla Python API.

### Proposed exercise

Try to implement sun position control similar to the reality.

![Sun position](./img/sun-position.png)

## Add actors to the simulation

### Coordinate system

Coordinates uses 3 axes: `x`, `y`, `z`, in meters.

There is also a rotation system with 3 angles: `pitch`, `yaw`, `roll`, in degrees.

In [ ]:
spectator = world.get_spectator()
spectator.set_transform(
    carla.Transform(
        carla.Location(x=10, y=10, z=10),
        carla.Rotation(yaw=5, pitch=0, roll=0)
    )
)

### Spawnpoints

Spawnpoints are used to place actors in the simulation, like vehicles, pedestrians, sensors, etc.

In [58]:
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
spectator = world.get_spectator()

for spawn_point in spawn_points:
    spectator.set_transform(spawn_point)

    print(spawn_point.location, end="\r")
    time.sleep(5)

KeyboardInterrupt: 

### Actors

In [59]:
blue_prints = world.get_blueprint_library().filter('*')

for blue_print in blue_prints:
    print(f"Id: {blue_print.id}, Tags: {blue_print.tags}")

Id: vehicle.nissan.micra, Tags: ['vehicle', 'nissan', 'micra']
Id: vehicle.audi.a2, Tags: ['vehicle', 'audi', 'a2']
Id: static.prop.plantpot04, Tags: ['static', 'prop', 'plantpot04']
Id: static.prop.bench02, Tags: ['static', 'bench02', 'prop']
Id: vehicle.mercedes.coupe_2020, Tags: ['vehicle', 'mercedes', 'coupe_2020']
Id: vehicle.audi.tt, Tags: ['vehicle', 'audi', 'tt']
Id: vehicle.ford.ambulance, Tags: ['vehicle', 'ford', 'ambulance']
Id: vehicle.bmw.grandtourer, Tags: ['vehicle', 'bmw', 'grandtourer']
Id: vehicle.harley-davidson.low_rider, Tags: ['vehicle', 'harley-davidson', 'low_rider']
Id: walker.pedestrian.0037, Tags: ['walker', '0037', 'pedestrian']
Id: vehicle.micro.microlino, Tags: ['vehicle', 'micro', 'microlino']
Id: vehicle.carlamotors.firetruck, Tags: ['vehicle', 'carlamotors', 'firetruck']
Id: walker.pedestrian.0038, Tags: ['walker', 'pedestrian', '0038']
Id: static.prop.trashcan05, Tags: ['static', 'prop', 'trashcan05']
Id: vehicle.carlamotors.carlacola, Tags: ['vehicle

### Spawn a vehicles

In [62]:
for x in range(len(blue_prints)-1):
    vehicle = world.try_spawn_actor(blue_prints[x], spawn_points[x])
    try:
        vehicle.set_autopilot(True)
    except:
        print("Impossibile spawnare")


Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile spawnare
Impossibile s

IndexError: list index out of range

In [63]:
vehicle.set_autopilot(True)

AttributeError: 'Actor' object has no attribute 'set_autopilot'

In [ ]:
vehicle.destroy()

True

: 